In [47]:
import numpy as np
import glob
import pandas as pd

In [48]:
#step 1 - combining all daily files into one file
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [49]:
# setting df as a DataFrame as combined_csv
#all data from the .csv is in strings it appears
df = pd.read_csv("combined_csv.csv")

#drop blanks from the ticker
df['ticker'] = df['ticker'].str.strip()
df['ticker'].replace('', np.nan, inplace=True)
df['ticker'].replace('TRUE', np.nan, inplace=True)
#drop ticker.str.len() > 6 from ticker (no ticker is longer than 6)
df['ticker'].values[df['ticker'].str.len() > 6,] = np.nan
df.dropna(subset=['ticker'], inplace=True)

#drop blanks and 0s from price
df['price'] = df['price'].str.strip() #strip blank spaces
df['price'].replace('', np.nan, inplace=True) #replace blanks with NaN
df['price'] = df['price'].replace(',','', regex=True) #strip commas
df['price'] = pd.to_numeric(df['price']) #convert price column from string to numbers
df['price'].replace(0,np.nan,inplace=True) #remove 0s
df.dropna(subset=['price'], inplace=True) #remove all NaNs

#converting num_mentions, pos_count, neg_count, price, price_change_net into int/floats
df['num_mentions'] = pd.to_numeric(df['num_mentions'])
df['pos_count'] = pd.to_numeric(df['pos_count'])
df['neg_count'] = pd.to_numeric(df['neg_count'])
df['price'] = pd.to_numeric(df['price'])
df['price_change_net'] = df['price_change_net'].replace(',','', regex=True) #strip commas
df['price_change_net'] = pd.to_numeric(df['price_change_net'])

#converting date column into date_time format
df['date'] = pd.to_datetime(df['date'])

#cleaning pct_mentions and converting to number
df['pct_mentions'] = df['pct_mentions'].str.strip() #strip blank spaces
df['pct_mentions'] = df['pct_mentions'].replace('<1','0.5', regex=True) #change '<1' to '0.5'
df['pct_mentions'] = pd.to_numeric(df['pct_mentions'])

# converting into floats then into % for relevant columns
#df['bullish_pct'] = df.price_change_pct.astype(float)
#df['bearish_pct'] = df.price_change_pct.astype(float)
#df['neutral_pct'] = df.price_change_pct.astype(float)
#df['pct_mentions'] = df.price_change_pct.astype(float)
df['bullish_pct'] = df.bullish_pct/100
df['bearish_pct'] = df.bearish_pct/100
df['neutral_pct'] = df.neutral_pct/100
df['pct_mentions'] = df.pct_mentions/100

In [50]:
#code from alex
# Convert this column to float
df.price_change_net = df.price_change_net.astype(float)

# Define a function to remove some characters in strings in a series
def remove_from_str(series, *args):
    for val in args:
        series = series.str.replace(val, '')
    
    return series

# Check rows that have price_change_pct == 'undefined'
df.price_change_pct = remove_from_str(df.price_change_pct, '+', ',', '%')

In [51]:
#check this again
df[df.price_change_pct == 'undefined']

,ticker,date,url,num_mentions,pct_mentions,pos_count,neg_count,bullish_pct,bearish_pct,neutral_pct,price,price_change_net,price_change_pct,time_of_price
5389,GRAMF,2021-01-19,https://finance.yahoo.com/quote/GRAMF?p=GRAMF,1,0.005,0,0,0.00,0.00,1.00,12.66,12.66,undefined,At close: 3:59PM EST
26552,COIN,2021-04-14,https://finance.yahoo.com/quote/COIN?p=COIN,131,0.050,30,37,0.22,0.28,0.48,250.00,250.00,undefined,At close: 8:00PM EDT
36348,NE,2021-06-10,https://finance.yahoo.com/quote/NE?p=NE,3,0.005,1,0,0.33,0.00,0.66,24.75,24.75,undefined,At close: 3:59PM EDT
36981,INDI,2021-06-11,https://finance.yahoo.com/quote/INDI?p=INDI,1,0.005,0,0,0.00,0.00,1.00,10.87,10.87,undefined,At close: 4:00PM EDT
37804,PTRA,2021-06-15,https://finance.yahoo.com/quote/PTRA?p=PTRA,2,0.005,0,0,0.00,0.00,1.00,18.09,18.09,undefined,At close: 3:59PM EDT
37946,MAPS,2021-06-16,https://finance.yahoo.com/quote/MAPS?p=MAPS,6,0.005,4,0,0.66,0.00,0.33,18.84,18.84,undefined,At close: 4:00PM EDT
40448,MMAT,2021-06-28,https://finance.yahoo.com/quote/MMAT?p=MMAT,11,0.005,5,0,0.45,0.00,0.54,9.90,9.90,undefined,At close: 4:00PM EDT
48321,RKLB,2021-08-25,https://finance.yahoo.com/quote/RKLB?p=RKLB,16,0.010,4,1,0.25,0.06,0.68,11.57,11.57,undefined,At close: 4:00PM EDT


In [52]:
# Only 8 rows, drop them
df = df[~(df.price_change_pct == 'undefined')]

# Convert to float
df['price_change_pct'] = df.price_change_pct.astype(float)

# Convert pct values to actual percents so it's not confusing (after we remove %)
df['price_change_pct'] = df.price_change_pct/100

In [8]:
# #ask alex about this later
# df.price_change_net = df.price_change_net.astype(float)
# df['price_change_pct'] = df['price_change_pct'].str.strip() #strip blank spaces
# df['price_change_pct'] = df['price_change_pct'].replace('%','', regex=True) #strip %
# df['price_change_pct'] = df['price_change_pct'].replace(',','', regex=True) #strip ,
# df['price_change_pct'] = df['price_change_pct'].replace('+','') #strip
# df['price_change_pct'] = df['price_change_pct'].replace('undefined','') #strip
# df.dropna(subset=['price_change_pct'], inplace=True)
# #df['price_change_pct'] = pd.to_numeric(df['price_change_pct']) #convert to integer

In [53]:
#set index to ticker
df = df.set_index('ticker')

In [54]:
df

,date,url,num_mentions,pct_mentions,pos_count,neg_count,bullish_pct,bearish_pct,neutral_pct,price,price_change_net,price_change_pct,time_of_price
ticker,,,,,,,,,,,,,
GME,2020-12-28,https://finance.yahoo.com/quote/GME?p=GME,567,0.200,257,131,0.45,0.23,0.31,20.99,0.84,0.0417,At close: 4:00PM EST
PLTR,2020-12-28,https://finance.yahoo.com/quote/PLTR?p=PLTR,418,0.140,164,110,0.39,0.26,0.34,25.63,-2.12,-0.0764,At close: 4:00PM EST
PSTH,2020-12-28,https://finance.yahoo.com/quote/PSTH?p=PSTH,80,0.020,51,19,0.63,0.23,0.12,25.99,-0.04,-0.0015,At close: 4:00PM EST
PSA,2020-12-28,https://finance.yahoo.com/quote/PSA?p=PSA,3,0.005,0,0,0.00,0.00,1.00,229.14,0.29,0.0013,At close: 4:00PM EST
TD,2020-12-28,https://finance.yahoo.com/quote/TD?p=TD,9,0.005,2,1,0.22,0.11,0.66,56.04,-0.04,-0.0007,At close: 4:00PM EST
...,...,...,...,...,...,...,...,...,...,...,...,...,...
BNGO,2021-10-08,https://finance.yahoo.com/quote/BNGO?p=BNGO,1,0.005,0,0,0.00,0.00,1.00,5.18,0.22,0.0444,At close: 4:00PM EDT
NNDM,2021-10-08,https://finance.yahoo.com/quote/NNDM?p=NNDM,1,0.005,0,0,0.00,0.00,1.00,5.68,0.16,0.0290,At close: 4:00PM EDT
COIN,2021-10-08,https://finance.yahoo.com/quote/COIN?p=COIN,3,0.005,3,0,1.00,0.00,0.00,251.59,1.21,0.0048,At close: 4:00PM EDT


In [55]:
df.dtypes

date                datetime64[ns]
url                         object
num_mentions                 int64
pct_mentions               float64
pos_count                    int64
neg_count                    int64
bullish_pct                float64
bearish_pct                float64
neutral_pct                float64
price                      float64
price_change_net           float64
price_change_pct           float64
time_of_price               object
dtype: object

In [56]:
# create new cleaned .csv file
df.to_csv('WSB.csv')